In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import numpy as np
import pandas as pd
import xarray as xr

from scipy import stats

import plotly.graph_objs as go
import cufflinks as cf

import ipywidgets as widgets
from ipywidgets import interact

cf.go_offline()

In [3]:
df_hmean = pd.read_csv("h_mean.csv.gz", index_col=0, header=[0,1])
df_fft = pd.read_parquet("fft.parquet.gzip", engine="pyarrow")
str_locs = df_fft.columns

ds = xr.load_dataset("fft_roll.nc")

# Hourly mean pressure

In [4]:
@interact

def show_hmean(loc=str_locs):
    
    df_hmean[loc].iplot(subplots=True, shape=(2,1), title=f"Location: {loc}")

interactive(children=(Dropdown(description='loc', options=('32.5,-120.0', '32.5,-97.5', '37.5,-122.5', '37.5,-…

# FFT results PSD plot
* Hourly mean signals were standardized (z score) before FFT
* PSD is normalized

In [5]:
@interact
def show_fft(loc=str_locs, logy=True):
    
    # chi 2 rv value for given ci
    rv = stats.chi2(df=2)
    chi95 = rv.ppf(0.95)
    chi99 = rv.ppf(0.99)

    # main plot
    fig = df_fft[loc].dropna().iplot(asFigure=True, logy=logy, xTitle="Freq [cpd]", yTitle="Normalized PSD", 
                                          color="Navy", title=f"Location: {loc}")
    # add horizontal line
    fig.add_trace(go.Scatter(x=[0, 12], y=[chi95, chi95], name="95% ci",
                             mode="lines", line=dict(color="Red",dash="dashdot")))
    
    fig.add_trace(go.Scatter(x=[0, 12], y=[chi99, chi99], name="99% ci",
                             mode="lines", line=dict(color="Orange",dash="dashdot")))
    
    # layout
    if logy:
        fig.update_layout(go.Layout(yaxis=dict(range=[-2,3])))
    
    
    fig.update_layout(showlegend=True)
    fig.show()

interactive(children=(Dropdown(description='loc', options=('32.5,-120.0', '32.5,-97.5', '37.5,-122.5', '37.5,-…

# Rolling fft PSD heat map

In [6]:
z_slider = widgets.FloatRangeSlider(value=(0,2), min=-8, max=4)

@interact

def show_fft_roll(loc=str_locs, colorscale="-purples", w=z_slider):
    
    # get as dataframe
    hm = ds[loc].to_dataframe().unstack(0).dropna().droplevel(0, axis=1)
    
    np.log10(hm).iplot(kind="heatmap", yTitle="Frequency [cpd]", 
                       title=f"Log 10 Nomralized PSD for location: {loc}", colorscale=colorscale, 
                       zmin=w[0], zmax=w[1])

interactive(children=(Dropdown(description='loc', options=('32.5,-120.0', '32.5,-97.5', '37.5,-122.5', '37.5,-…

## Color scales
Tip: add "-" before color scale name to get inverted colorscale

In [7]:
sc = cf.colors.scales()